In [1]:
from collections import Counter
import json
import os
import string
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader

In [2]:
namedf = pd.read_csv("surnames_with_splits.csv")

,nationality,nationality_index,split,surname
0,Arabic,15,train,Totah
1,Arabic,15,train,Abboud
2,Arabic,15,train,Fakhoury
3,Arabic,15,train,Srour
4,Arabic,15,train,Sayegh
...,...,...,...,...
10975,Vietnamese,11,test,Dinh
10976,Vietnamese,11,test,Phung
10977,Vietnamese,11,test,Quang
10978,Vietnamese,11,test,Vu


In [4]:
class LMDataset(Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self,dataframe):
        
                
        #self.X = ...
        #self.Y = ...
        
    def vectorize_name(self,trigram):
        
        #return ...
    
    def __len__(self):
        'Denotes the total number of samples'
        #return len(self.X)

    def __getitem__(self, index):
        'Generates one sample of data'
       
        x = self.X[index]
        y = self.Y[index]
        
        return x,y

def get_loader(dataframe, batch_size=16, shuffle=True):
    'returns a DataLoader instance with the given parameters'
    dataset = LMDataset(dataframe)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


IndentationError: expected an indented block after function definition on line 3 (407421450.py, line 9)

In [ ]:
class TrigramLMOneHot(nn.Module):

    def __init__(self, hidden_dim, vocab_dim):
       
        super(TrigramLMOneHot, self).__init__()
        
        self.hidden_layer = nn.Linear(vocab_dim*3,hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, vocab_dim)
        self.sm = nn.Softmax(dim=1) # softmax über output nodes/spalten deswegen dim=1    

    def forward(self, x_in):
        
        x = self.hidden_layer(x_in)
        x = self.output_layer(x)
        
        return self.sm(x) 
    
    
 # https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html   

In [ ]:
trilm = TrigramLMOneHot(10,2)

In [ ]:
trial_x = torch.tensor([[0.,1.,1.,0.,0.,1.]])

In [ ]:
trilm(trial_x)

tensor([[0.4252, 0.5748]], grad_fn=<SoftmaxBackward>)

In [ ]:
list(trilm.hidden_layer.parameters())

[Parameter containing:
 tensor([[-0.0158, -0.0628,  0.3137,  0.0865,  0.0375, -0.3028],
         [ 0.3559,  0.1028, -0.2616,  0.1874, -0.3832,  0.1978],
         [-0.0655, -0.3617, -0.1292,  0.2155,  0.1781,  0.0625],
         [ 0.1485,  0.3845, -0.1732,  0.1940,  0.1259,  0.3541],
         [ 0.3923, -0.1236, -0.0380, -0.0767, -0.2842,  0.1664],
         [-0.3430,  0.2940, -0.2840, -0.2097, -0.3190,  0.3057],
         [-0.0241, -0.0845,  0.1926, -0.0720,  0.4026, -0.0905],
         [-0.1535,  0.3562,  0.2219,  0.0493, -0.3052,  0.3850],
         [-0.0145,  0.3299, -0.2035, -0.3052,  0.2326, -0.1636],
         [ 0.1593, -0.1346,  0.3602, -0.0137, -0.4006,  0.0246]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.2550, -0.1711,  0.0713,  0.3085, -0.1851, -0.3092, -0.0190, -0.2625,
          0.1267,  0.2011], requires_grad=True)]

In [ ]:
lr = 0.1
optimizer = optim.Adam(params=trilm.parameters(), lr=lr)
ce_loss = nn.NLLLoss()
epoch = 0  # reset epoch counter
n_epochs = 3

In [ ]:
#For each epoch
for epoch in range(n_epochs):
    print('Epoch : ', epoch)
    sum_loss=0
    
    #For each batch
    for batch_index, (x,y) in enumerate(train_loader):
        #print(x, x.type(), x.size())
    
        #reset gradient
        optimizer.zero_grad()
        
        #prepare the x as tensor
        
        #run the model, get the prediction for x
        y_pred = trilm(x)
        
        #print(y_pred)
        
        #compare it with label, calculate loss, add it to epoch loss
        loss=ce_loss(y_pred, y)
        sum_loss+=loss.item()
        
        #calculate gradients
        loss.backward()
        
        #call optimizer to update the weights backwards 
        optimizer.step()
        
    print('Loss : ', sum_loss)

In [ ]:
def vectorize_trigram(trigram,start=False):
    
    if start:
        return [vocab['START']]+[vocab[char] for char in trigram]

    return [vocab[char] for char in trigram]


def generate_char(model, bigram_prefix,sample=False):
    #sample_generated_text= prefix
    #greedy_generated_text= prefix
    
    
    prefix_generated = bigram_prefix
    
    while len(prefix_generated) < 15:
        print(prefix_generated)
        model.eval()
        if len(prefix) < 3:
            prefix_ids = torch.tensor(vectorize_trigram(prefix_generated,start=True))
        else:
            prefix_ids = torch.tensor(vectorize_trigram(prefix_generated,start=False))
        inp=prefix_ids[-3:]
        
        #print('last 3 chars', inp, 'shape:', inp.size)
        y_pred=model(inp)
        #print(y_pred, len(y_pred))
        #print(y_pred.exp(), len(y_pred))
        
        if sample:
            top1 = torch.multinomial(y_pred.exp(),1).detach().numpy()[0][0]
        else:
            top1 = torch.argmax(y_pred, dim=1).detach().numpy()[0]
            #print(top_1)
        
        
        prefix_generated += idx_to_token[top1]
        print(prefix_generated)
        
        if idx_to_token[top1] == "END":
            break
    
    print("generated:",prefix_generated)
    return  prefix_generated
        